<a href="https://www.kaggle.com/code/shiyamaladevirs/final-obdtrackerid?scriptVersionId=226988534" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/demodataset/Week 10 NikhilkrishnanOBD1.xlsx
/kaggle/input/demodataset/Copy of OBD Sensitive Verticals _ A (1) (1).xlsx


**PLEASE USE GPU T4 x2 For FASTER RESULTS**

In [5]:
!pip install XlsxWriter

In [6]:
import time
import os
import re
import csv
import pandas as pd
import numpy as np

# Try to import cuDF for GPU acceleration; if not available, fall back to Dask.
try:
    import cudf
    gpu_available = True
    print("GPU acceleration available: Using RAPIDS cuDF")
except ImportError:
    gpu_available = False
    import dask.dataframe as dd
    print("GPU acceleration not available: Using Dask")

def read_xlsx_as_dataframe(file):
    """
    Reads an XLSX file using openpyxl in read-only mode
    and converts the data to a pandas DataFrame.
    """
    from openpyxl import load_workbook
    wb = load_workbook(filename=file, read_only=True, data_only=True)
    ws = wb.active
    data = list(ws.values)
    # Assume first row contains column headers.
    columns = data[0]
    df = pd.DataFrame(data[1:], columns=columns)
    return df

def load_main_file(main_file, npartitions=4):
    """
    Load the main file (CSV or XLSX) as a DataFrame.
    Uses cuDF if GPU is available; otherwise, uses Dask.
    For XLSX files, uses openpyxl's read-only mode to improve performance.
    """
    ext = os.path.splitext(main_file)[1].lower()
    if gpu_available:
        if ext == '.csv':
            print("Reading main CSV file with cuDF on GPU...")
            df = cudf.read_csv(main_file)
        elif ext in ['.xlsx', '.xls']:
            print("Reading main XLSX file using openpyxl (read-only mode) and converting to cuDF on GPU...")
            df_pd = read_xlsx_as_dataframe(main_file)
            # Convert object columns to string to avoid MixedTypeError.
            for col in df_pd.select_dtypes(include=['object']).columns:
                df_pd[col] = df_pd[col].astype(str)
            df = cudf.DataFrame.from_pandas(df_pd)
        else:
            raise ValueError(f"Unsupported file format: {ext}")
    else:
        if ext == '.csv':
            print("Reading main CSV file with Dask...")
            df = dd.read_csv(
                main_file,
                engine='python',
                sep=',',
                quoting=csv.QUOTE_NONE,
                escapechar='\\',
                on_bad_lines='skip',
                encoding='latin1',
                assume_missing=True,
                blocksize="100MB"
            )
        elif ext in ['.xlsx', '.xls']:
            print("Reading main XLSX file using openpyxl (read-only mode) and converting to Dask DataFrame...")
            df_pd = read_xlsx_as_dataframe(main_file)
            df = dd.from_pandas(df_pd, npartitions=npartitions)
        else:
            raise ValueError(f"Unsupported file format: {ext}")
    return df

def load_obd_sensitive(obd_file):
    """
    Load OBD sensitive verticals from an Excel file.
    If the 'vertical' column is missing, the first column is used.
    Returns a set of normalized sensitive keywords.
    """
    obd_df = pd.read_excel(obd_file, engine='openpyxl')
    if 'vertical' not in obd_df.columns:
        print("Column 'vertical' not found in OBD file. Using the first column as 'vertical'.")
        first_col = obd_df.columns[0]
        obd_df['vertical'] = obd_df[first_col]
    obd_df['vertical'] = obd_df['vertical'].astype(str).str.strip().str.lower()
    return set(obd_df['vertical'].dropna())

def ensure_columns(pdf):
    """
    Ensure required columns exist in each partition.
    """
    if 'hub_zone' not in pdf.columns:
        pdf['hub_zone'] = "unknown"
    return pdf

def process_dataframe(df, combined_sensitive):
    """
    Process the dataframe:
      - It first creates a temporary combined column (from brand, product_title, vertical)
        to filter out rows matching sensitive keywords.
      - It captures the removed rows (i.e. those matching OBD sensitive verticals).
      - Then it further filters out rows if hub_name contains "myntra" or hub_type contains "kirana".
      - It also fills missing 'parent_lm_hub' and hub_zone values.
    Returns a tuple:
      (filtered_df, removed_obd_sensitive_df, filtering_stats)
    where filtering_stats is a dictionary with counts of rows before/after filtering and per filter.
    """
    # Strip whitespace from column names but preserve original case
    df.columns = df.columns.str.strip()
    
    # For Dask, ensure every partition has the necessary columns.
    if not gpu_available:
        df = df.map_partitions(ensure_columns)
    
    # Strip whitespace for key text columns (without converting to lower-case)
    text_cols = ['brand', 'product_title', 'vertical', 'hub_name', 'parent_lm_hub', 'hub_zone', 'hub_type']
    for col in text_cols:
        if col in df.columns:
            df[col] = df[col].astype(str).str.strip()
    
    if not gpu_available:
        df = df.reset_index(drop=True)
    
    # Record initial row count.
    rows_before = len(df) if gpu_available else df.shape[0].compute()
    
    # -------------------------------
    # OBD Sensitive Filtering
    # -------------------------------
    removed_obd_count = 0
    removed_obd_sensitive = None
    required_cols = ['brand', 'product_title', 'vertical']
    if all(c in df.columns for c in required_cols):
        # Create temporary combined column (in lower-case)
        df['temp_combined'] = (df['brand'].fillna('') + " " + 
                                 df['product_title'].fillna('') + " " + 
                                 df['vertical'].fillna('')).str.lower().str.strip()
        pattern = '|'.join([re.escape(word) for word in combined_sensitive])
        if gpu_available:
            mask_sensitive = df['temp_combined'].str.contains(pattern)
            removed_obd_sensitive = df[mask_sensitive].copy()
            removed_obd_count = len(removed_obd_sensitive)
            df = df[~mask_sensitive]
        else:
            mask_sensitive = df['temp_combined'].str.contains(pattern, case=False, na=False)
            removed_obd_sensitive = df[mask_sensitive].copy()
            removed_obd_count = df[mask_sensitive].shape[0].compute()
            df = df[~mask_sensitive]
        df = df.drop(columns=['temp_combined'])
    else:
        removed_obd_sensitive = None

    # -------------------------------
    # Filter out rows where hub_name contains "myntra"
    # -------------------------------
    removed_myntra_count = 0
    if 'hub_name' in df.columns:
        if gpu_available:
            mask_myntra = df['hub_name'].fillna('').str.lower().str.contains("myntra", regex=False)
            removed_myntra_count = len(df[mask_myntra])
            df = df[~mask_myntra]
        else:
            mask_myntra = df['hub_name'].str.contains("myntra", case=False, na=False)
            removed_myntra_count = df[mask_myntra].shape[0].compute()
            df = df[~mask_myntra]
    
    # -------------------------------
    # Filter out rows where hub_type contains "kirana"
    # -------------------------------
    removed_kirana_count = 0
    if 'hub_type' in df.columns:
        if gpu_available:
            mask_kirana = df['hub_type'].fillna('').str.lower().str.contains("kirana", regex=False)
            removed_kirana_count = len(df[mask_kirana])
            df = df[~mask_kirana]
        else:
            mask_kirana = df['hub_type'].str.contains("kirana", case=False, na=False)
            removed_kirana_count = df[mask_kirana].shape[0].compute()
            df = df[~mask_kirana]
    
    rows_after = len(df) if gpu_available else df.shape[0].compute()
    
    # -------------------------------
    # Fill missing values for parent_lm_hub and hub_zone
    # -------------------------------
    if 'hub_name' in df.columns and 'parent_lm_hub' in df.columns:
        df['parent_lm_hub'] = df['parent_lm_hub'].replace(
            {"": np.nan, "unknown": np.nan, "null": np.nan, "nan": np.nan, "n/a": np.nan}
        ).fillna(df['hub_name'])
    
    def fill_hub_zone(series):
        valid = series[~series.isin(["", "unknown", "null", "nan", "n/a"])]
        fill_value = valid.iloc[0] if not valid.empty else "unknown"
        return series.fillna(fill_value).replace(["", "unknown", "null", "nan", "n/a"], fill_value)
    
    if 'hub_name' in df.columns and 'hub_zone' in df.columns:
        if gpu_available:
            df_pd = df.to_pandas()
            df_pd['hub_zone'] = df_pd.groupby('hub_name')['hub_zone'].transform(fill_hub_zone)
            df = cudf.DataFrame.from_pandas(df_pd)
        else:
            df = df.shuffle(on="hub_name")
            df['hub_zone'] = df.groupby('hub_name')['hub_zone'].transform(fill_hub_zone)
    
    filtering_stats = {
        "rows_before": rows_before,
        "rows_after": rows_after,
        "removed_obd_sensitive": removed_obd_count,
        "removed_myntra": removed_myntra_count,
        "removed_kirana": removed_kirana_count,
        "total_removed": rows_before - rows_after
    }
    
    return df, removed_obd_sensitive, filtering_stats

def sample_per_hub(df, n=5):
    """
    Sample up to n rows per 'parent_lm_hub' group.
    If 'parent_lm_hub' does not exist, fallback to grouping by 'hub_name'.
    """
    def sample_group(pdf, n=n):
        return pdf.sample(n=n, random_state=42) if len(pdf) > n else pdf
    
    if 'parent_lm_hub' in df.columns:
        group_col = 'parent_lm_hub'
    elif 'hub_name' in df.columns:
        group_col = 'hub_name'
    else:
        if gpu_available:
            return df.to_pandas()
        else:
            return df.compute()
    
    if gpu_available:
        df_pd = df.to_pandas()
        sampled_pd = df_pd.groupby(group_col).apply(lambda x: sample_group(x, n)).reset_index(drop=True)
        return sampled_pd
    else:
        sampled_ddf = df.groupby(group_col).apply(sample_group, meta=df._meta)
        final_df = sampled_ddf.compute()
        return final_df

def main(main_file, obd_file, output_file):
    overall_start = time.time()
    
    # Load OBD sensitive keywords.
    start = time.time()
    obd_verticals = load_obd_sensitive(obd_file)
    manual_sensitive = {'pampers', 'diaper'}  # Additional keywords.
    combined_sensitive = obd_verticals.union(manual_sensitive)
    print("Combined sensitive keywords:", combined_sensitive)
    print("Time to load OBD sensitive file: {:.2f} seconds".format(time.time() - start))
    
    # Load main data.
    start = time.time()
    df_loaded = load_main_file(main_file)
    print("Time to load main file: {:.2f} seconds".format(time.time() - start))
    
    # Compute distinct hubs count before filtering.
    if 'parent_lm_hub' in df_loaded.columns:
        if gpu_available:
            distinct_hubs_before = df_loaded['parent_lm_hub'].nunique()
        else:
            distinct_hubs_before = df_loaded['parent_lm_hub'].nunique().compute()
    elif 'hub_name' in df_loaded.columns:
        if gpu_available:
            distinct_hubs_before = df_loaded['hub_name'].nunique()
        else:
            distinct_hubs_before = df_loaded['hub_name'].nunique().compute()
    else:
        distinct_hubs_before = "N/A"
    
    # Process the dataframe (filtering).
    start = time.time()
    df_filtered, removed_obd_sensitive, filtering_stats = process_dataframe(df_loaded, combined_sensitive)
    print("Time to process dataframe: {:.2f} seconds".format(time.time() - start))
    
    # Compute distinct hubs count after filtering.
    if 'parent_lm_hub' in df_filtered.columns:
        if gpu_available:
            distinct_hubs_after = df_filtered['parent_lm_hub'].nunique()
        else:
            distinct_hubs_after = df_filtered['parent_lm_hub'].nunique().compute()
    elif 'hub_name' in df_filtered.columns:
        if gpu_available:
            distinct_hubs_after = df_filtered['hub_name'].nunique()
        else:
            distinct_hubs_after = df_filtered['hub_name'].nunique().compute()
    else:
        distinct_hubs_after = "N/A"
    
    # Update overview stats.
    overview_stats = {
        "Total rows before filtering": filtering_stats["rows_before"],
        "Total rows after filtering": filtering_stats["rows_after"],
        "Total rows removed": filtering_stats["total_removed"],
        "Removed due to OBD sensitive": filtering_stats["removed_obd_sensitive"],
        "Removed due to myntra": filtering_stats["removed_myntra"],
        "Removed due to kirana": filtering_stats["removed_kirana"],
        "Distinct hubs before filtering": distinct_hubs_before,
        "Distinct hubs after filtering": distinct_hubs_after
    }
    
    # Sample final data per hub.
    start = time.time()
    final_df = sample_per_hub(df_filtered, n=5)  # Sampling 5 rows per parent_lm_hub group.
    print("Time to sample per hub: {:.2f} seconds".format(time.time() - start))
    
    # Write to Excel with multiple sheets.
    start = time.time()
    with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
        # Write the final tracking data.
        final_df.to_excel(writer, sheet_name="TrackingIDs", index=False)
        # Write the removed OBD sensitive rows.
        if removed_obd_sensitive is not None:
            if gpu_available:
                removed_obd_df = removed_obd_sensitive.to_pandas()
            else:
                removed_obd_df = removed_obd_sensitive.compute()
            removed_obd_df.to_excel(writer, sheet_name="OBD Removed", index=False)
        else:
            # If nothing was removed, write an empty DataFrame.
            pd.DataFrame().to_excel(writer, sheet_name="OBD Removed", index=False)
        # Write the overview statistics.
        overview_df = pd.DataFrame(list(overview_stats.items()), columns=["Metric", "Value"])
        overview_df.to_excel(writer, sheet_name="Overview", index=False)
    print("Time to write Excel: {:.2f} seconds".format(time.time() - start))
    
    overall_time = time.time() - overall_start
    print("Total time taken: {:.2f} seconds".format(overall_time))
    print("Done. Output written to", output_file)

if __name__ == '__main__':
    MAIN_FILE = "/kaggle/input/demodataset/Week 10 NikhilkrishnanOBD1.xlsx"  # or .xlsx file
    OBD_FILE = "/kaggle/input/demodataset/Copy of OBD Sensitive Verticals _ A (1) (1).xlsx"
    OUTPUT_FILE = "/kaggle/working/processed_data.xlsx"
    main(MAIN_FILE, OBD_FILE, OUTPUT_FILE)


GPU acceleration available: Using RAPIDS cuDF
Column 'vertical' not found in OBD file. Using the first column as 'vertical'.
Combined sensitive keywords: {'menstrunk', 'mensvest', 'sexualmassager', 'breastnipplecare', 'boysleepwear', 'adultdiapers', 'breastpump', 'girlsleepwear', 'femaledisorders', 'pregnancykit', 'tampon', 'womenboxer', 'womennightdressnighty', 'pantyliner', 'girlinnerwear', 'womenlingerieset', 'mensswimsuit', 'pampers', 'boyinnerwear', 'girlswimsuit', 'condom', 'feedingnursing', 'sanitarypad', 'fertilitysupplement', 'babybooty', 'womenswimsuit', 'womencamisoleslip', 'femaleurinationdevice', 'womenbra', 'infantbodysuit', 'infantinnerwear', 'sexualhealth', 'womenintimatecare', 'womennightsuit', 'womenpanty', 'girlblouse', 'mensbrief', 'womenshirttoptunic', 'infantsleepwear', 'womensportbra', 'roleplaytoy', 'mensboxer', 'fertilitykit', 'womenbabydoll', 'menstrualcups', 'pleasureenhancement', 'maledisorders', 'womenshapewear', 'girlbodysuit', 'sexualcomboandkit', 'diaper

<ipython-input-6-3f26c00d08fd>:236: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_pd = df_pd.groupby(group_col).apply(lambda x: sample_group(x, n)).reset_index(drop=True)


Time to sample per hub: 5.41 seconds
Time to write Excel: 30.93 seconds
Total time taken: 227.10 seconds
Done. Output written to /kaggle/working/processed_data.xlsx
